In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [2]:
SYSTEM_PROMPT = """
You are the Router for a restaurant assistant. Classify the user's message into exactly one intent:
- menu: questions about food/menu items, categories, veg/non-veg, ingredients, allergens, prices, recommendations, ratings, combos.
- about: questions about the restaurant itself: name, location/address, opening/closing hours, contact/phone, story/vision/mission, ambience.
- faqs: policies/amenities: parking, Wi-Fi, payment methods, pet/kid friendliness, wheelchair access, smoking area, delivery partners, refunds/returns, general facilities.
- escalate: requests to place/modify/cancel orders, make reservations/table bookings, payment/transaction issues, urgent complaints requiring staff.
- none: greetings, thanks, small talk, jokes, or generic conversation you can handle directly without tools.

Rules:
- Prefer escalate if the user asks to "book", "reserve", "order", "pay", "cancel", or similar operational requests.
- Prefer menu when the focus is food details, even if price/rating is mentioned.
- Prefer about for hours, address, contact, or the restaurant’s story.
- Prefer faqs for policies and facilities.
- Use none for chit-chat (hi/hello/thanks/ok/lol/good night) or general praise.

Return ONLY JSON in one line:
{"intent":"menu|about|faqs|escalate|none","confidence":0.0-1.0,"rationale":"<short reason>"}
Keep rationale under 12 words.
"""


In [10]:
from typing import TypedDict

class State(TypedDict):
    user_query: str
    query_intent: str

In [14]:
def classify_intent(state: State) -> State:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"User: {state['user_query']}\nReturn only the JSON."},
    ]
    resp = llm.invoke(messages)
    state["query_intent"] = resp.content
    return state


In [19]:
from langgraph.graph import StateGraph, START, END
builder = StateGraph(State)
builder.add_node("classify", classify_intent)
builder.add_edge(START, "classify")
builder.add_edge("classify", END)
graph = builder.compile()

In [23]:
# demo
if __name__ == "__main__":
    tests = [
        "hi there",
        "do you take orders?",
        "what's the veg options that you have?",
        "i want something refreshing",
        "who are you",
        "what's buff choila",
    ]
    for t in tests:
        print(t, "->", graph.invoke({"user_query": t}))

hi there -> {'user_query': 'hi there', 'query_intent': '{"intent":"none","confidence":1.0,"rationale":"User is providing a simple greeting."}'}
do you take orders? -> {'user_query': 'do you take orders?', 'query_intent': '{"intent":"escalate","confidence":1.0,"rationale":"User asks about placing orders, which requires staff intervention."}'}
what's the veg options that you have? -> {'user_query': "what's the veg options that you have?", 'query_intent': '{"intent":"menu","confidence":1.0,"rationale":"User asks about vegetarian food options from the menu."}'}
i want something refreshing -> {'user_query': 'i want something refreshing', 'query_intent': '{"intent":"menu","confidence":1.0,"rationale":"User wants a refreshing item, which relates to menu items."}'}
who are you -> {'user_query': 'who are you', 'query_intent': '{"intent":"none","confidence":1.0,"rationale":"User asks a direct question about the AI\'s identity, which is general chat."}'}
what's buff choila -> {'user_query': "what